In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import sys

sys.path.append("/nas/ckgfs/users/minhpham/workspace/seed")
sys.path.append("/nas/ckgfs/users/minhpham/workspace/BLINK")

In [10]:
from seed.pipeline.pipeline import Pipeline

In [11]:
from seed.pipeline.retriever import Retriever

retriever = Retriever.get("wiki")

/nas/home/minhpham/miniconda3/envs/ml11/lib/python3.8/site-packages/huggingface_hub/file_download.py:560: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2022-07-23 22:23:19,067 loading file /nas/home/minhpham/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-07-23 22:23:21,925 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [12]:
from seed.pipeline.verifier import Verifier

verifier = Verifier.get("bart")

In [13]:
from seed.pipeline.extractor import Extractor

extractor = Extractor.get("unifiedqa")

/nas/home/minhpham/miniconda3/envs/ml11/lib/python3.8/site-packages/huggingface_hub/file_download.py:560: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2022-07-23 22:23:36,902 loading file /nas/home/minhpham/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-07-23 22:23:39,750 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [15]:
pipeline = Pipeline(retriever, verifier, extractor)

In [8]:
import pandas as pd
import jsonlines

df = pd.DataFrame(list(jsonlines.open("../data/totto/test.jsonl")))

In [14]:
import json

df["pred"] = None
df["is_correct"] = False

for idx, obj in df.iterrows():
    table = pd.DataFrame([obj["table"]])
    label = obj["label"]
    column = pipeline.run(table)
    print(obj)

    if not label:
        _, _, _, wrong_value = obj["counter_fact"]
    else:
        wrong_value = None
    df.loc[idx, "is_correct"] = table[column][0] == wrong_value
    
    if column is not None:
        df.loc[idx, "preds"] = table[column][0]
    else:
        df.loc[idx, "preds"] = None

Charles L . McNary ( R
Alexander Smith
tensor([ 0.5959,  2.3620, -2.2211], grad_fn=<SelectBackward0>)
tensor([-0.0223,  3.2536, -2.7105], grad_fn=<SelectBackward0>)
tensor([-0.5520,  3.3724, -1.9055], grad_fn=<SelectBackward0>)
tensor([ 0.1432,  4.2737, -3.5683], grad_fn=<SelectBackward0>)
tensor([-0.4541,  4.7583, -3.4388], grad_fn=<SelectBackward0>)
tensor([ 1.1912,  3.1846, -3.7095], grad_fn=<SelectBackward0>)
tensor([-0.1318,  3.8344, -3.7192], grad_fn=<SelectBackward0>)
tensor([ 0.1288,  4.3818, -3.6169], grad_fn=<SelectBackward0>)
tensor([-0.6065,  4.5285, -3.0937], grad_fn=<SelectBackward0>)
tensor([-1.4653,  5.3535, -2.8533], grad_fn=<SelectBackward0>)
tensor([-1.2427,  5.3549, -3.2404], grad_fn=<SelectBackward0>)


KeyboardInterrupt: 